## Demo 2 - Ordering a set of ducks

source: https://www.cs.columbia.edu/CAVE/software/softlib/coil-20.php 

In [ ]:
# Imports
import numpy as np

# For image and video manipulation
from PIL import Image
import cv2  # if not installed, use : !pip install opencv-python
import glob

# Persistent homology
from ripser import ripser
from persim import plot_diagrams
from dreimac import CircularCoords

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Auxiliary video functions
def writeVideo(filename, frame_data, fps, resol ):
    n_row, n_col = resol
    out = cv2.VideoWriter(filename, cv2.VideoWriter_fourcc(*'mp4v'), fps, (n_col, n_row), False)
    for i in range(len(frame_data)):
        out.write(frame_data[i].reshape(n_row, n_col) )
    out.release()

def playVideo(filepath):
    cap = cv2.VideoCapture(filepath)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cv2.imshow('frame', gray)
        if cv2.waitKey(50) == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()   

In [ ]:
# Load and display the image data

data = []
img_names = glob.glob('data/Duck_Data/*.png')

for filename in img_names:
    img = np.asarray(Image.open(filename))
    data.append(img.reshape(-1))
    
data = np.array(data)
np.random.shuffle(data) # randomly permute the image order

plt.figure(figsize = (12,2))
for i in range(72):
    plt.subplot(4, 18, i + 1)
    plt.imshow(data[i].reshape(128,128), cmap='gray')
    plt.axis('off')

In [ ]:
# Create a video from the image data. Frames are ordered in the same way as in the data set
writeVideo('ducks_original.mp4', data, 12, (128,128))

In [ ]:
# Play the video. A new window should open
playVideo('ducks_original.mp4')

In [ ]:
# Rips persistence of the data

res = ripser(data, maxdim=2,  coeff =2 )
dgms = res['dgms']
plt.figure(figsize = (4,4)) 
plot_diagrams(dgms)

In [ ]:
# Compute the circular coordinates of the data
cc = CircularCoords(data , distance_matrix = False, n_landmarks  = 72, prime = 13)

# Get the circular coordinate at interpolation scale = 0.9, and for the cocycle with highest persistence
theta = cc.get_coordinates(perc= 0.2,  cocycle_idx=[0])

In [ ]:
# Sort the images with respect to the computed circular coordinate
data_sort = data[np.argsort(theta)]

# Display the sorted data
plt.figure(figsize = (12,2))
for i in range(72):
    plt.subplot(4, 18, i + 1)
    plt.imshow(data_sort[i].reshape(128,128), cmap='gray')
    plt.axis('off')
    

In [ ]:
# Create a video from the image data. Frames are ordered by their circular coordinate
writeVideo('ducks_ordered.mp4', data_sort, 12, (128,128))

In [ ]:
# Play the video. A new window should open
playVideo('ducks_ordered.mp4')